
## Imports and Dependencies
This section includes all the libraries required for the progra (simulation and dashboard) m:

- `heapq`: Used to manage priority queues for operational and charging queues of Robotinos.
- `random`: Simulates random events such as battery drainage amounts.
- `time`: Introduces delays in the simulation to mimic real-world timing.
- `Thread`: Manages the simulation as a separate thread.
- `Dash`, `html`, `dcc`, `Input`, `Output`: Used to create a dynamic dashboard for visualizing the Robotino simulation.
- `dash_bootstrap_components`: Adds Bootstrap-styled components to the Dash dashboard.
- `plotly.graph_objects`: Used to create interactive graphs for data visualization in the dashboard.


In [ ]:
import heapq
import random
import time
from threading import Thread
from dash import Dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go

## Class Definitions

### Robotino
This class represents a robot with the following attributes and methods:
- **Attributes**:
  - `robot_id`: A unique identifier for the Robotino.
  - `version`: The version of the Robotino, which determines compatibility with chargers.
  - `status`: The operational status of the Robotino (e.g., `operational`, `charging`).
  - `battery_level`: Represents the Robotino's battery percentage.

- **Methods**:
  - `drain_battery(amount)`: Reduces the battery level by a given amount.
  - `recharge_battery()`: Recharges the battery by 16%, up to a maximum of 100%.
  - `set_status(new_status)`: Updates the Robotino's status.
  - Special methods like `__str__` and `__lt__` provide string representation and comparison logic for sorting by battery level.


In [ ]:
class Robotino:
    def __init__(self, robot_id, version, initial_status):
        self.robot_id = robot_id
        self.version = version
        self.status = initial_status
        self.battery_level = 100  # Start with a full battery

    def __str__(self):
        return f"Robotino-{self.robot_id} (Version {self.version}, {self.status}, Battery: {self.battery_level}%)"

    def __lt__(self, other):
        return self.battery_level < other.battery_level

    def drain_battery(self, amount):
        self.battery_level = max(0, self.battery_level - amount)

    def recharge_battery(self):
        """Increase the battery level by 20%, up to a maximum of 100%."""
        if self.battery_level < 100:
            self.battery_level = min(100, self.battery_level + 16)

    def set_status(self, new_status):
        self.status = new_status

### Charger
This class represents a charging station for Robotinos with the following:
- **Attributes**:
  - `charger_id`: A unique identifier for the charger.
  - `compatible_version`: The Robotino version that the charger supports.
  - `current_robotino`: Tracks the Robotino currently connected to the charger.

- **Methods**:
  - `connect(robotino)`: Connects a compatible Robotino to the charger and updates its status to `charging`.
  - `disconnect()`: Disconnects the currently charging Robotino from the charger.


In [ ]:
class Charger:
    def __init__(self, charger_id, compatible_version):
        self.charger_id = charger_id
        self.compatible_version = compatible_version
        self.current_robotino = None

    def __str__(self):
        status = "Available" if self.current_robotino is None else f"Charging {self.current_robotino}"
        return f"Charger-{self.charger_id} (Compatible with Version {self.compatible_version}, {status})"

    def connect(self, robotino):
        if self.current_robotino is None and robotino.version == self.compatible_version:
            self.current_robotino = robotino
            robotino.set_status("charging")
            return True
        return False

    def disconnect(self):
        if self.current_robotino:
            self.current_robotino = None

## System Initialization
In this section, the system initializes the following components:

1. **Queues**:
   - `operational_queue`: Priority queue for Robotinos ready to perform tasks.
   - `charging_queue`: Priority queue for Robotinos waiting to charge.

2. **Chargers**:
   - Six chargers are created. Charger-1 is compatible with version 4, and the rest are compatible with version 3.

3. **Robotinos**:
   - Six Robotinos are created with predefined IDs, versions, and initial statuses (either `operational` or `charging`).

4. **Tasks**:
   - A dictionary of 10 tasks is created, along with a mapping of tasks to Robotino IDs.


In [ ]:
operational_queue = []
charging_queue = []
chargers = [Charger(i + 1, compatible_version=4 if i == 0 else 3) for i in range(6)]

robotinos = [Robotino(robot_id=i + 1, version=4 if i == 0 else 3,
                      initial_status="operational" if i < 3 else "charging")
             for i in range(6)]

for robotino in robotinos[:3]:
    heapq.heappush(operational_queue, robotino)
for robotino in robotinos[3:]:
    heapq.heappush(charging_queue, robotino)

tasks = {i: f"Task-{i}" for i in range(1, 11)}
robotino_tasks = {robotino.robot_id: None for robotino in robotinos}


## Core Functions

### Assign and Unassign Tasks
These functions manage task assignments for Robotinos:

- **`assign_task(robotino)`**:
  - Assigns the first available task from the `tasks` dictionary to the specified Robotino.
  - Updates the `tasks` dictionary to mark the task as unavailable.

- **`unassign_task(robotino)`**:
  - Removes the task assigned to a Robotino.
  - Updates the `tasks` dictionary to mark the task as available again.


In [ ]:
def assign_task(robotino):
    for task_id, task in tasks.items():
        if task:
            robotino_tasks[robotino.robot_id] = task
            tasks[task_id] = None
            return

def unassign_task(robotino):
    task = robotino_tasks[robotino.robot_id]
    if task:
        for task_id, task_name in tasks.items():
            if task_name is None:
                tasks[task_id] = task
                break
        robotino_tasks[robotino.robot_id] = None

### Charger Interaction
This section provides functions for managing Robotino-charger interactions:

- **`find_available_charger(robotino)`**:
  - Returns the first available charger compatible with the specified Robotino's version.

- **`swap_robotinos()`**:
  - Simulates Robotino operations, battery drainage, and task assignments.
  - Handles the transfer of Robotinos between the operational and charging queues based on their battery levels.
  - Robotinos with low battery are removed from the operational queue and placed in the charging queue.
  - Fully charged Robotinos are moved back to the operational queue, assigned a new task, and slightly drained to simulate readiness.


In [ ]:
def find_available_charger(robotino):
    for charger in chargers:
        if charger.compatible_version == robotino.version and charger.current_robotino is None:
            return charger
    return None

def swap_robotinos():
    """Simulates Robotino operation, battery drainage, and task swapping."""
    drained_robotinos = []
    for robotino in operational_queue:
        robotino.drain_battery(random.randint(10, 12))  # Drain battery by a random amount
        if robotino.battery_level < 10:  # Needs charging
            drained_robotinos.append(robotino)

    for low_battery_robotino in drained_robotinos:
        unassign_task(low_battery_robotino)  # Unassign task before charging
        operational_queue.remove(low_battery_robotino)
        heapq.heapify(operational_queue)

        charger = find_available_charger(low_battery_robotino)
        if charger:
            charger.connect(low_battery_robotino)
            heapq.heappush(charging_queue, low_battery_robotino)
        else:
            heapq.heappush(operational_queue, low_battery_robotino)  # Stay operational if no charger

    while len(operational_queue) < 3 and charging_queue:
        ready_robotino = heapq.nlargest(1, charging_queue)[0]
        charging_queue.remove(ready_robotino)
        heapq.heapify(charging_queue)

        for charger in chargers:
            if charger.current_robotino == ready_robotino:
                charger.disconnect()
                break

        ready_robotino.set_status("operational")
        ready_robotino.drain_battery(5)  # Small initial drain
        heapq.heappush(operational_queue, ready_robotino)
        assign_task(ready_robotino)

## Simulation Logic
This section defines the main simulation loop:

- **`run_simulation()`**:
  - Simulates 30 cycles of Robotino operation.
  - Each cycle performs the following:
    1. **Swap Robotinos**:
       - Moves Robotinos between operational and charging queues based on their battery levels.
    2. **Recharge Batteries**:
       - Incrementally charges Robotinos in the charging queue.
    3. **Display Status**:
       - Outputs the state of batteries, queues, chargers, and tasks to the console.
    4. **Cycle Delay**:
       - Introduces a 5-second delay between cycles to mimic real-time operation.


In [ ]:
def run_simulation():
    for cycle in range(30):  # Simulate 30 cycles
        print(f"\nCycle {cycle + 1}:")
        print("=" * 30)
        swap_robotinos()
        for robotino in charging_queue:
            robotino.recharge_battery()

        # Display current states
        display_battery_levels(robotinos)
        display_charger_status(chargers)
        display_queue_states(operational_queue, charging_queue)
        display_tasks(robotino_tasks)
        time.sleep(5)  # Wait for 5 seconds before next cycle


## Dash Web Dashboard
The Dash web dashboard visualizes the simulation in real time:

1. **Layout**:
   - Displays graphs for battery levels, queue sizes, and charger statuses.
   - Shows a table of task assignments.

2. **Callback**:
   - Updates the dashboard every 5 seconds using a callback function.
   - The callback fetches data from the simulation and renders it in graphs and tables.


In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dcc.Interval(id="interval-component", interval=5 * 1000, n_intervals=0),
    dbc.Row(html.H1("Robotino Simulation Dashboard", className="text-center mb-4")),
    dbc.Row([
        dbc.Col(dcc.Graph(id="battery-levels"), width=6),
        dbc.Col(html.Div(id="task-assignments"), width=6)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id="queue-states"), width=6),
        dbc.Col(dcc.Graph(id="charger-status"), width=6)
    ])
])

@app.callback(
    Output("battery-levels", "figure"),
    Output("task-assignments", "children"),
    Output("queue-states", "figure"),
    Output("charger-status", "figure"),
    Input("interval-component", "n_intervals")
)
def update_dashboard(n_intervals):
    battery_levels = [r.battery_level for r in robotinos]
    robot_ids = [f"R-{r.robot_id}" for r in robotinos]
    battery_fig = go.Figure(go.Bar(x=robot_ids, y=battery_levels, marker_color="blue"))
    task_display = html.Table(children=[
        html.Tr([html.Th("Robot"), html.Th("Task")]),
        *[html.Tr([html.Td(robot_ids[i]), html.Td(robotino_tasks.get(robotinos[i].robot_id, "None"))])
          for i in range(len(robotinos))]
    ], className="table table-hover")
    queue_fig = go.Figure(go.Bar(
        x=["Operational Queue", "Charging Queue"],
        y=[len(operational_queue), len(charging_queue)],
        marker_color=["green", "purple"]
    ))
    charger_status = [c.current_robotino.robot_id if c.current_robotino else "Empty" for c in chargers]
    charger_ids = [f"C-{c.charger_id}" for c in chargers]
    charger_fig = go.Figure(go.Bar(x=charger_ids, y=[1 if s != "Empty" else 0 for s in charger_status], marker_color="cyan"))
    return battery_fig, task_display, queue_fig, charger_fig

if __name__ == "__main__":
    print("Starting simulation. Visit the dashboard at http://127.0.0.1:8080/")
    app.run_server(debug=True, port=8080)


### Minor Lines Of code Unexplained
### Display Functions
These helper functions output the state of the simulation to the console:
- **Battery Levels**: Displays each Robotino's battery percentage.
- **Charger Status**: Indicates whether chargers are available or occupied.
- **Queue States**: Lists Robotinos in operational and charging queues.
- **Task Assignments**: Maps tasks to Robotinos.


In [ ]:
def display_battery_levels(robotinos):
    print("\nBattery Levels:")
    for robotino in robotinos:
        print(f"  {robotino}")

def display_charger_status(chargers):
    print("\nCharger Status:")
    for charger in chargers:
        print(f"  {charger}")

def display_queue_states(operational_queue, charging_queue):
    print("\nQueue States:")
    print(f"  Operational Queue: {[r.robot_id for r in operational_queue]}")
    print(f"  Charging Queue: {[r.robot_id for r in charging_queue]}")

def display_tasks(robotino_tasks):
    print("\nTask Assignments:")
    for robot_id, task in robotino_tasks.items():
        task_display = task if task else "None"
        print(f"  Robotino-{robot_id}: {task_display}")


### Simulation Thread
The simulation logic (`run_simulation`) is executed in a separate thread using the `Thread` module. This allows the simulation to run concurrently with the Dash dashboard.
- **`daemon=True`**: Ensures the thread stops when the main program exits.


In [ ]:
simulation_thread = Thread(target=run_simulation, daemon=True)
simulation_thread.start()


### Dash Callback
The Dash callback updates the dashboard every 5 seconds (`interval-component`) with the latest data from the simulation:
- **Inputs**:
  - `n_intervals`: A counter that increments with each interval, triggering updates.
- **Outputs**:
  - `battery-levels`: A bar graph showing battery levels of Robotinos.
  - `task-assignments`: A table displaying the task assigned to each Robotino.
  - `queue-states`: A bar graph representing the sizes of operational and charging queues.
  - `charger-status`: A bar graph indicating whether chargers are occupied or available.


### Simulation Timing
The simulation includes a `5-second` delay (`time.sleep(5)`) between each cycle:
- **Purpose**:
  - Simulates real-world operation timing.
  - Allows time for users to view updated states in the console or dashboard before the next cycle begins.
